#### Read Data

In [18]:
import pandas as pd
import requests
import gzip
import shutil
import os

os.makedirs('data', exist_ok=True)
url = "http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data.gz"
compressed_file = "data/kddcup.data.gz"
data_file = "data/kddcup.data.csv"

response = requests.get(url, stream=True)
with open(compressed_file, 'wb') as f:
    f.write(response.content)

with gzip.open(compressed_file, 'rb') as f_in:
    with open(data_file, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [20]:
import pandas as pd
column_names = [
    "duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes",
    "land", "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in",
    "num_compromised", "root_shell", "su_attempted", "num_root",
    "num_file_creations", "num_shells", "num_access_files", "num_outbound_cmds",
    "is_host_login", "is_guest_login", "count", "srv_count", "serror_rate",
    "srv_serror_rate", "rerror_rate", "srv_rerror_rate", "same_srv_rate",
    "diff_srv_rate", "srv_diff_host_rate", "dst_host_count", "dst_host_srv_count",
    "dst_host_same_srv_rate", "dst_host_diff_srv_rate", "dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate", "dst_host_serror_rate", "dst_host_srv_serror_rate",
    "dst_host_rerror_rate", "dst_host_srv_rerror_rate", "label"
]
data_file = 'data/kddcup.data.csv'
df = pd.read_csv(data_file, header=None, names=column_names)
df = df.drop('num_outbound_cmds', axis=1)
df.head()

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,label
0,0,tcp,http,SF,215,45076,0,0,0,0,...,0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,162,4528,0,0,0,0,...,1,1.0,0.0,1.00,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,236,1228,0,0,0,0,...,2,1.0,0.0,0.50,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,233,2032,0,0,0,0,...,3,1.0,0.0,0.33,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,239,486,0,0,0,0,...,4,1.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,normal.


#### Cleaning/Preprocessing

(The cleaning steps below are deduced from insights gained from extensive visualizations and data wrangling steps, which have been omitted from the notebook for the sake of cleanliness)

In [21]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

# Identify feature types
categorical_features = ['protocol_type', 'service', 'flag']
binary_features = ['land', 'logged_in', 'is_host_login', 'is_guest_login']
numeric_features = [col for col in df.columns if col not in categorical_features + binary_features + ['label']]

# Split the data into train and test sets
X = df.drop('label', axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [22]:
# Define preprocessing pipelines
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('bin', binary_transformer, binary_features)
    ]
)

In [23]:
X_train_preprocessed = preprocessor.fit_transform(X_train)

X_test_preprocessed = preprocessor.transform(X_test)

# Convert the preprocessed features back to DataFrames
# Get feature names after one-hot encoding
cat_feature_names = preprocessor.named_transformers_['cat']['onehot'].get_feature_names_out(categorical_features)
all_feature_names = numeric_features + list(cat_feature_names) + binary_features

df_train_cleaned = pd.DataFrame(X_train_preprocessed, columns=all_feature_names)
df_test_cleaned = pd.DataFrame(X_test_preprocessed, columns=all_feature_names)

# Add the target variable back to the DataFrames
df_train_cleaned['label'] = y_train.values
df_test_cleaned['label'] = y_test.values

In [24]:
df_train_cleaned.describe()

,duration,src_bytes,dst_bytes,wrong_fragment,urgent,hot,num_failed_logins,num_compromised,root_shell,su_attempted,...,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,land,logged_in,is_host_login,is_guest_login
count,3.918744e+06,3.918744e+06,3.918744e+06,3.918744e+06,3.918744e+06,3.918744e+06,3.918744e+06,3.918744e+06,3.918744e+06,3.918744e+06,...,3.918744e+06,3.918744e+06,3.918744e+06,3.918744e+06,3.918744e+06,3.918744e+06,3.918744e+06,3.918744e+06,3.918744e+06,3.918744e+06
mean,-9.793039e-18,4.487645e-20,-8.997955e-19,5.122262e-18,-4.193002e-19,-1.294074e-17,-1.387090e-19,7.819382e-20,1.095167e-18,2.137751e-18,...,1.775684e-01,1.046254e-04,3.368426e-05,1.148327e-05,7.643796e-01,2.112922e-04,5.358860e-06,1.434807e-01,5.103676e-07,8.359821e-04
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,3.821490e-01,1.022812e-02,5.803717e-03,3.388679e-03,4.243860e-01,1.453436e-02,2.314915e-03,3.505624e-01,7.144001e-04,2.890127e-02
min,-6.681397e-02,-1.967301e-03,-3.745250e-03,-1.514353e-02,-9.810075e-04,-2.651866e-02,-4.387833e-03,-2.015437e-03,-8.285479e-03,-4.596510e-03,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,-6.681397e-02,-1.925493e-03,-3.745250e-03,-1.514353e-02,-9.810075e-04,-2.651866e-02,-4.387833e-03,-2.015437e-03,-8.285479e-03,-4.596510e-03,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,-6.681397e-02,-1.473205e-03,-3.745250e-03,-1.514353e-02,-9.810075e-04,-2.651866e-02,-4.387833e-03,-2.015437e-03,-8.285479e-03,-4.596510e-03,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,-6.681397e-02,-9.867117e-04,-3.745250e-03,-1.514353e-02,-9.810075e-04,-2.651866e-02,-4.387833e-03,-2.015437e-03,-8.285479e-03,-4.596510e-03,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,5.934995e+01,1.311217e+03,1.972283e+03,7.002059e+01,1.855876e+03,1.641068e+02,6.613338e+02,1.772033e+03,1.206931e+02,2.450641e+02,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


#### Experimenting with Models